# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [22]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [23]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [24]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [31]:
#Produto escolhido:
produto = 'volkswagen'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [32]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [33]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [290]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

v = pd.read_excel('volkswagen.xlsx')

xls = pd.ExcelFile('volkswagen.xlsx')
df1 = xls.parse('Treinamento')
df2 = xls.parse('Teste')


In [291]:
IR = []
irrelevante =[]
R =[]
relevante = []
for linha in v[v.Relevancia == "irrelevante"].Treinamento:
    IR.extend(linha.split())
for i in IR:
    if len(i)>3:
        irrelevante.append(i)
for linha in v[v.Relevancia != "irrelevante"].Treinamento:
    R.extend(linha.split())
for i in R:
    if len(i)>3:
        relevante.append(i)

In [292]:
Test=[]

for i in df2.Teste:
    si = str(i)
    if len(si)>3:
        Test.append(si)

In [293]:
listaP =[]
Out = []

TIR=pd.value_counts(irrelevante)


for linha in Test:
    linhas = linha.split()
    for palavra in linhas:                           
        if palavra in irrelevante:
            P = TIR[palavra]/len(irrelevante)    #P de ser irrelevante
            listaP.append(P)
        
        else:
            n = 1/ len(irrelevante)  #Se for uma palavra que ainda não foi catalogada
            listaP.append(n)        
       
        Pr = 0 
        for x in listaP:  #Soma
            Pr += x
           
        if Pr>0.5:
            Pro = "irrelevante"
        if Pr < 0.5 :
            Pro = "relevante"
    
    Out.append(Pro)
            
    #print(listaP)      
#print(Out)  

In [294]:
df2

,Teste,Relevancia
0,#felizsemana caen ventas 13.4 % de volkswagen ...,irrelevante
1,@volkswagen_mx feliz domingo!,irrelevante
2,volkswagen anuncia recall de quase 2 milhões d...,irrelevante
3,rt @skacush: figa do lixo eh a volkswagen http...,relevante
4,"rt @prosapolitica: renault, ford, volkswagen, ...",relevante
5,rt @_francomica: volkswagen gol power 1.6 2008...,irrelevante
6,se vende volkswagen gol https://t.co/zks3oosgm...,irrelevante
7,#operacaobetalab #timbeta #beta #betalab #beta...,irrelevante
8,volkswagen reativa terceiro turno em sua fábri...,irrelevante
9,a história do @vwbrasil fusca em texto de 2009...,irrelevante


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [295]:
Res=[]
FP=0
FN=0
VP=0
VN=0
r=0
ir=0

for linha in range (len(Out)):          ##Compara com os resultados que foram classificados
    if Out[linha] == df2.Relevancia[linha]:
        if Out[linha] == "relevante": 
            res = "VP"
            VP+=1
            r+=1
        if Out[linha] == "irrelevante":
            res = "VN"    
            VN+=1
            ir+=1
    if Out[linha] != df2.Relevancia[linha]:
        if Out[linha] == "relevante":
            res = "FP"
            FP+=1
            r+=1
        if Out[linha] == "irrelevante":
            res = "FN"
            FN+=1
            ir+=1
    Res.append(res)
#print(Res)

In [296]:
PFP = FP/len(Res)*100  #Porcentagens sobre o total
PFN = FN/len(Res)*100
PVP = VP/len(Res)*100
PVN = VN/len(Res)*100

FPP = FP/r*100         #Porcentagens sobre Positivos/Negativos
FNN = FN/ir*100
VPP = VP/r*100
VNN = VN/ir*100
print("FP",PFP)
print("FN",PFN)
print("VP",PVP)
print("VN",PVN,"\n")

print("FP/P",FPP)
print("FN/N",FNN)
print("VP/P",VPP)
print("VN/N",VNN)

FP 2.0
FN 7.5
VP 1.0
VN 89.5 

FP/P 66.66666666666666
FN/N 7.731958762886598
VP/P 33.33333333333333
VN/N 92.26804123711341


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


O classificador não foi muito bom para classificar tweets relevantes, porém teve cerca de 92% de precisão ao classificar tweets como irrelevantes, assim talvez fosse mais efetivo como um filtro anti-spam, por exemplo. Parte de sua imprecisão se deve ao fato de ser difícil detectar recursos linguísticos como o uso sarcasmo ou dupla negação, isso se deve a como é feita a classificação: as palavras de tweets irrelevantes são armazenadas e é calculada qual a probabilidade, dada as palavras de um tweet ainda não classificado, dele ser relevante ou não, assim uma mensagem que tenha várias palavras em comum com outras que sejam consideradas irrelevante, terá grandes chances de também ser classificada dessa forma.<br />
Portanto, o projeto pode ser expandido levando em conta a semântica das frases, assim haverá uma maior precisão nas classificações uma vez que as palavras não vão ser analisadas individualmente, mas seu efeito no sentido da frase como um todo
